In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments

In [2]:
# Load Train, Validation, Test Dataset
train_dataset = pd.read_csv('../saveDir/LOS_WEEKS_adm_train.csv')
val_dataset = pd.read_csv('../saveDir/LOS_WEEKS_adm_val.csv')
test_dataset = pd.read_csv('../saveDir/LOS_WEEKS_adm_test.csv')

In [3]:
# from transformers import AutoModelForSequenceClassification, AutoConfig

# # Specify the dropout rate in the configuration
# config = AutoConfig.from_pretrained('bvanaken/CORe-clinical-outcome-biobert-v1', 
#                                     num_labels=4, 
#                                     hidden_dropout_prob=0.2, 
#                                     attention_probs_dropout_prob=0.2)

# # Load the pre-trained model with the specified configuration
# core_model = AutoModelForSequenceClassification.from_pretrained('bvanaken/CORe-clinical-outcome-biobert-v1', config=config)


In [4]:
# from transformers import AutoModelForSequenceClassification, AutoConfig

# # Specify the dropout rate in the configuration
# config = AutoConfig.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', 
#                                     num_labels=4, 
#                                     hidden_dropout_prob=0.2, 
#                                     attention_probs_dropout_prob=0.2)

# # Load the pre-trained model with the specified configuration
# clinical_model = AutoModelForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', config=config)

In [5]:
# from transformers import AutoModelForSequenceClassification, AutoConfig

# # Specify the dropout rate in the configuration
# config = AutoConfig.from_pretrained('dmis-lab/biobert-base-cased-v1.2', 
#                                     num_labels=4, 
#                                     hidden_dropout_prob=0.2, 
#                                     attention_probs_dropout_prob=0.2)

# # Load the pre-trained model with the specified configuration
# base_model = AutoModelForSequenceClassification.from_pretrained('dmis-lab/biobert-base-cased-v1.2', config=config)

In [6]:
from transformers import AutoModelForSequenceClassification, AutoConfig

# Specify the dropout rate in the configuration
config = AutoConfig.from_pretrained('emilyalsentzer/Bio_Discharge_Summary_BERT', 
                                    num_labels=4, 
                                    hidden_dropout_prob=0.2, 
                                    attention_probs_dropout_prob=0.2)

# Load the pre-trained model with the specified configuration
discharge_model = AutoModelForSequenceClassification.from_pretrained('emilyalsentzer/Bio_Discharge_Summary_BERT', config=config)

Some weights of the model checkpoint at emilyalsentzer/Bio_Discharge_Summary_BERT were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from

In [7]:
from transformers import AutoTokenizer

# Choose a tokenizer
tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_Discharge_Summary_BERT')

In [8]:
from transformers import BertTokenizer, BertModel
import torch

def split_inputs(input_ids, attention_mask, tokenizer):
    # Get the special token ids
    cls_token_id = tokenizer.cls_token_id
    sep_token_id = tokenizer.sep_token_id

    # Split the tensors
    input_ids1 = input_ids[:, :510]
    input_ids2 = input_ids[:, 460:970]

    attention_mask1 = attention_mask[:, :510]
    attention_mask2 = attention_mask[:, 460:970]

    # Add [CLS] and [SEP] tokens
    input_ids1 = torch.cat([torch.full_like(input_ids1[:, :1], cls_token_id), input_ids1, torch.full_like(input_ids1[:, :1], sep_token_id)], dim=1)
    input_ids2 = torch.cat([torch.full_like(input_ids2[:, :1], cls_token_id), input_ids2, torch.full_like(input_ids2[:, :1], sep_token_id)], dim=1)

    attention_mask1 = torch.cat([torch.full_like(attention_mask1[:, :1], 1), attention_mask1, torch.full_like(attention_mask1[:, :1], 1)], dim=1)
    attention_mask2 = torch.cat([torch.full_like(attention_mask2[:, :1], 1), attention_mask2, torch.full_like(attention_mask2[:, :1], 1)], dim=1)

    return (input_ids1, attention_mask1), (input_ids2, attention_mask2)

In [9]:
from torch.utils.data import DataLoader
from torch import nn

class EnsembleModel(nn.Module):
    def __init__(self, model1):
        super(EnsembleModel, self).__init__()
        self.model1 = model1

    def forward(self, input_ids, attention_mask):
        # Assuming your batch size (18) tensors are input_ids and attention_mask
        (input_ids1, attention_mask1), (input_ids2, attention_mask2) = split_inputs(input_ids, attention_mask, tokenizer)

        outputs1 = self.model1(input_ids1, attention_mask=attention_mask1)[0]
        outputs2 = self.model1(input_ids2, attention_mask=attention_mask2)[0]
        
        avg_output = (outputs1 + outputs2) / 2.00
        return avg_output

In [10]:
# Apply the tokenizer to the training, validation, and test datasets
train_encodings = tokenizer(train_dataset['text'].tolist(), truncation=True, padding=True, max_length = 970)
val_encodings = tokenizer(val_dataset['text'].tolist(), truncation=True, padding=True,  max_length = 970)
test_encodings = tokenizer(test_dataset['text'].tolist(), truncation=True, padding=True , max_length = 970)

In [11]:
# Create a Dataset for PyTorch
class LosDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [12]:
train_dataset = LosDataset(train_encodings, train_dataset['los_label'].tolist())
val_dataset = LosDataset(val_encodings, val_dataset['los_label'].tolist())
test_dataset = LosDataset(test_encodings, test_dataset['los_label'].tolist())

In [13]:
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, accuracy_score
from tqdm import tqdm
from torch import nn
import numpy as np

# Create the ensemble model
ensemble_model = EnsembleModel(discharge_model)

In [14]:
import os

# list all files in the current directory
files = os.listdir('.')

# filter the ones that start with 'CORE_baseline'
core_models = [f for f in files if f.startswith('dischargeBERT_chunk_over')]

if core_models:
    print("Found models starting with 'dischargeBERT_chunk_over':")
    for model in core_models:
        print(model)
        
    # get the first (and supposedly only) model
    model_path = core_models[0]

    # load the model state
    ensemble_model.load_state_dict(torch.load(model_path))
    print(f"Loaded Model{model_path}")
else:
    print("No models found starting with 'dischargeBERT_chunk_over'.")

Found models starting with 'dischargeBERT_chunk_over':
dischargeBERT_chunk_over_epoch_0roc_0.6132276433822398.pth
Loaded ModeldischargeBERT_chunk_over_epoch_0roc_0.6132276433822398.pth


In [15]:
# ensemble_model

In [16]:
# Push the model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ensemble_model = ensemble_model.to(device)

In [17]:
train_loader = DataLoader(train_dataset, batch_size=9, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=9, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=9, shuffle=False)

In [19]:
epochs = 200
best_roc_auc = 0.613227
min_delta = 0.0001
early_stopping_count = 0
early_stopping_patience = 3
gradient_accumulation_steps = 10

# Set the optimizer
optimizer = AdamW(ensemble_model.parameters(), lr=1e-5, weight_decay=0.01)

# Set the scheduler
scheduler = get_linear_schedule_with_warmup(
    optimizer, 
    num_warmup_steps=50, 
    num_training_steps=len(train_loader) * epochs // gradient_accumulation_steps
)


In [19]:
# device = "cpu"

In [ ]:
from torch.nn import functional as F
# Training
for epoch in range(epochs):
    ensemble_model.train()
    train_loss = 0
    for step, batch in enumerate(tqdm(train_loader)):
        optimizer.zero_grad() if step % gradient_accumulation_steps == 0 else None
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        (loss / gradient_accumulation_steps).backward()
        train_loss += loss.item()
        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_loader):
            optimizer.step()
            scheduler.step()

    ensemble_model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = ensemble_model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            val_loss += loss.item()
            val_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
            val_labels.append(labels.cpu().numpy())
            

    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    val_loss /= len(val_loader)
    train_loss /= len(train_loader)
    print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss}')

    # Calculate metrics
    val_preds_class = np.argmax(val_preds, axis=1)
    accuracy = accuracy_score(val_labels, val_preds_class)
    recall = recall_score(val_labels, val_preds_class, average='weighted')
    precision = precision_score(val_labels, val_preds_class, average='weighted')
    f1 = f1_score(val_labels, val_preds_class, average='weighted')
    micro_f1 = f1_score(val_labels, val_preds_class, average='micro')
    macro_roc_auc = roc_auc_score(val_labels, val_preds, multi_class='ovo', average='macro')

    print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Micro F1: {micro_f1}, Macro Roc Auc: {macro_roc_auc}')
        
        
    # Implement early stopping
    if epoch > 0 and macro_roc_auc - best_roc_auc < min_delta:
        early_stopping_count += 1
        print(f'EarlyStopping counter: {early_stopping_count} out of {early_stopping_patience}')
        if early_stopping_count >= early_stopping_patience:
            print('Early stopping')
            break
    else:
        best_roc_auc = macro_roc_auc
        early_stopping_count = 0
        torch.save(ensemble_model.state_dict(), f"dischargeBERT_chunk_over_epoch_{epoch}roc_{best_roc_auc}.pth")
        

100%|██████████| 488/488 [04:26<00:00,  1.83it/s]
/home/ec2-user/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 1/200, Training Loss: 1.3091510367513373, Validation Loss: 1.2866108108495102
Accuracy: 0.381006604418128, Recall: 0.381006604418128, Precision: 0.32048196166456844, F1: 0.3401967456763074, Micro F1: 0.38100660441812795, Macro Roc Auc: 0.6132276433822398


 42%|████▏     | 1416/3381 [35:35<50:15,  1.53s/it] 

In [20]:
from torch.nn import functional as F
# Training
for epoch in range(epochs):
    ensemble_model.train()
    train_loss = 0
    for step, batch in enumerate(tqdm(train_loader)):
        optimizer.zero_grad() if step % gradient_accumulation_steps == 0 else None
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        (loss / gradient_accumulation_steps).backward()
        train_loss += loss.item()
        if (step + 1) % gradient_accumulation_steps == 0 or (step + 1) == len(train_loader):
            optimizer.step()
            scheduler.step()

    ensemble_model.eval()
    val_loss = 0
    val_preds = []
    val_labels = []
    with torch.no_grad():
        for batch in tqdm(val_loader):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)
            outputs = ensemble_model(input_ids, attention_mask)
            loss = nn.CrossEntropyLoss()(outputs, labels)
            val_loss += loss.item()
            val_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
            val_labels.append(labels.cpu().numpy())
            

    val_preds = np.concatenate(val_preds)
    val_labels = np.concatenate(val_labels)
    val_loss /= len(val_loader)
    train_loss /= len(train_loader)
    print(f'Epoch: {epoch+1}/{epochs}, Training Loss: {train_loss}, Validation Loss: {val_loss}')

    # Calculate metrics
    val_preds_class = np.argmax(val_preds, axis=1)
    accuracy = accuracy_score(val_labels, val_preds_class)
    recall = recall_score(val_labels, val_preds_class, average='weighted')
    precision = precision_score(val_labels, val_preds_class, average='weighted')
    f1 = f1_score(val_labels, val_preds_class, average='weighted')
    micro_f1 = f1_score(val_labels, val_preds_class, average='micro')
    macro_roc_auc = roc_auc_score(val_labels, val_preds, multi_class='ovo', average='macro')

    print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Micro F1: {micro_f1}, Macro Roc Auc: {macro_roc_auc}')
        
        
    # Implement early stopping
    if epoch > 0 and macro_roc_auc - best_roc_auc < min_delta:
        early_stopping_count += 1
        print(f'EarlyStopping counter: {early_stopping_count} out of {early_stopping_patience}')
        if early_stopping_count >= early_stopping_patience:
            print('Early stopping')
            break
    else:
        best_roc_auc = macro_roc_auc
        early_stopping_count = 0
        torch.save(ensemble_model.state_dict(), f"dischargeBERT_chunk_over_epoch_{epoch}roc_{best_roc_auc}.pth")
        

100%|██████████| 488/488 [04:25<00:00,  1.84it/s]
/home/ec2-user/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1268: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Epoch: 1/200, Training Loss: 1.2489219011727326, Validation Loss: 1.2600475630310715
Accuracy: 0.3873832839899795, Recall: 0.3873832839899795, Precision: 0.33982959782805167, F1: 0.3614771568192891, Micro F1: 0.3873832839899795, Macro Roc Auc: 0.6573414538313801


100%|██████████| 488/488 [04:05<00:00,  1.98it/s]


Epoch: 2/200, Training Loss: 1.211146007078493, Validation Loss: 1.247687294468528
Accuracy: 0.40901844682304717, Recall: 0.40901844682304717, Precision: 0.44772743920629365, F1: 0.3812290659267437, Micro F1: 0.4090184468230472, Macro Roc Auc: 0.6798412944395151


100%|██████████| 488/488 [04:08<00:00,  1.96it/s]


...

Epoch: 8/200, Training Loss: 1.0451992644744665, Validation Loss: 1.2057715344135878
Accuracy: 0.4500113869278069, Recall: 0.4500113869278069, Precision: 0.4473678375912117, F1: 0.43587093348483014, Micro F1: 0.4500113869278069, Macro Roc Auc: 0.7176999401130897
EarlyStopping counter: 1 out of 3


100%|██████████| 488/488 [04:25<00:00,  1.84it/s]


Epoch: 9/200, Training Loss: 1.0142790753922142, Validation Loss: 1.222453139844488
Accuracy: 0.4570712821680711, Recall: 0.4570712821680711, Precision: 0.45973435523626743, F1: 0.44485322004568006, Micro F1: 0.4570712821680711, Macro Roc Auc: 0.7159696635785336
EarlyStopping counter: 2 out of 3


100%|██████████| 488/488 [04:21<00:00,  1.86it/s]

Epoch: 10/200, Training Loss: 0.9829144789265726, Validation Loss: 1.2382412886277574
Accuracy: 0.4390799362332043, Recall: 0.4390799362332043, Precision: 0.44206697688446167, F1: 0.437109299894854, Micro F1: 0.4390799362332043, Macro Roc Auc: 0.7113587871758001
EarlyStopping counter: 3 out of 3
Early stopping


In [21]:
import os

# list all files in the current directory
files = os.listdir('.')

# filter the ones that start with 'CORE_baseline'
core_models = [f for f in files if f.startswith('dischargeBERT_chunk_over')]

if core_models:
    print("Found models starting with 'dischargeBERT_chunk_over':")
    for model in core_models:
        print(model)
        
    # get the first (and supposedly only) model
    model_path = core_models[0]

    # load the model state
    ensemble_model.load_state_dict(torch.load(model_path))
    print("Loaded Model")
else:
    print("No models found starting with 'dischargeBERT_chunk_over'.")

Found models starting with 'dischargeBERT_chunk_over':
dischargeBERT_chunk_over_epoch_6roc_0.7196560411068053.pth
Loaded Model


In [22]:
# Put the model in evaluation mode
ensemble_model.eval()

# Initialize lists to store predictions and true labels
test_preds = []
test_labels = []

# Iterate over test data
with torch.no_grad():
    for batch in tqdm(test_loader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        outputs = ensemble_model(input_ids, attention_mask)
        test_preds.append(F.softmax(outputs, dim=1).cpu().numpy())
        test_labels.append(labels.cpu().numpy())



100%|██████████| 978/978 [08:39<00:00,  1.88it/s]


In [23]:
test_preds = np.concatenate(test_preds)
test_labels = np.concatenate(test_labels)

# Calculate metrics
test_preds_class = np.argmax(test_preds, axis=1)
accuracy = accuracy_score(test_labels, test_preds_class)
recall = recall_score(test_labels, test_preds_class, average='weighted')
precision = precision_score(test_labels, test_preds_class, average='weighted')
f1 = f1_score(test_labels, test_preds_class, average='weighted')
micro_f1 = f1_score(test_labels, test_preds_class, average='micro')
macro_roc_auc = roc_auc_score(test_labels, test_preds, multi_class='ovo', average='macro')

print(f'Accuracy: {accuracy}, Recall: {recall}, Precision: {precision}, F1: {f1}, Micro F1: {micro_f1}, Macro Roc Auc: {macro_roc_auc}')

Accuracy: 0.44787995907695805, Recall: 0.44787995907695805, Precision: 0.44788409204595997, F1: 0.4442533527029775, Micro F1: 0.44787995907695805, Macro Roc Auc: 0.7277744754727147
